In [49]:
import re
import math
import matplotlib.pyplot as plt
from collections import Counter

In [50]:
unigrams = {}
bigrams = {}
testUnigrams = {}
testBigrams = {}

with open('English_train.txt', 'r', encoding='utf8') as f:
    train_lines = f.readlines()

with open('English_test.txt', 'r', encoding='utf8') as f:
    test_lines = f.readlines()

In [51]:
def tokenize(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower())

In [52]:
def word_ngrams(sent, n):
    """Given a sent as str return n-grams as a list of tuple"""

    tp=()
    listtp= []
    token = tokenize(sent)
    #if it is not 1-gram append <s> to the begining and </s> at the end
    if(n!=1):
        token.insert(0,"<s>")
        token.append("</s>")
    
    #generate n-grams using the zip function in python
    ngrams = zip(*[token[i:] for i in range(n)])
    
    #loop through the n-grams generated and create list of tuples
    for ngram in ngrams:
        listtp.append(ngram)
        
    return listtp


In [53]:
#helper funcion fr ngrams
def getGram(lines, n):
  bigramst=[]
  if type(lines) == list:
    for sentences in lines:
      bg=word_ngrams(sentences, n)
      bigramst+=bg
  else:
    bigramst = word_ngrams(lines, n)
  return bigramst

In [54]:
v_unigrams = getGram(train_lines, 1)
v_bigrams = getGram(train_lines, 2)
unigrams=Counter(v_unigrams)
bigrams=Counter(v_bigrams)

tv_unigrams = getGram(test_lines, 1)
tv_bigrams = getGram(test_lines, 2)
testUnigrams=Counter(tv_unigrams)
testBigrams=Counter(tv_bigrams)


In [55]:
def calculcate_unigram_prob(word, alpha = 0.3):
    N = len(wordTokens)
    V = len(unigrams)
    word_count = unigrams[word]

    unigram_prob = (word_count + alpha)/((alpha * V)+ N)
    return unigram_prob

def calculcate_bigram_prob(word1, word2, alpha = 0.3):
    w1_count = unigrams[word1]
    V = len(unigrams)
    bigram_count = bigrams[(word1, word2)] 
    bigram_prob = (bigram_count + alpha)/((alpha * V)+ w1_count)
    return bigram_prob

def estimate_interpolated_bigram_prob(word1, word2, lamda1 = 0.5, lamda2 = 0.5):
    prob_w2 = calculcate_unigram_prob(word2)
    prob_w1w2 = calculcate_bigram_prob(word1, word2)
    interpolated_bigram_prob =   (lamda1 * prob_w2) + (lamda2 * prob_w1w2)
    return interpolated_bigram_prob


In [57]:
def compute_perplexity(interp = True):
    sumlog = 0
    for bigram in testBigrams:
        word1 = bigram[0]
        word2 = bigram[1]
        if interp:
            bigram_prob = estimate_interpolated_bigram_prob(word1, word2)
        else:
            bigram_prob = calculcate_bigram_prob(word1, word2)
        log_bigram_prob = math.log(bigram_prob, 2)
        sumlog = sumlog + log_bigram_prob
    perplexity = pow(2, (-1) * sumlog/len(testBigrams))  
    return perplexity          
    
print("Perplexity with Interpolation: ", compute_perplexity(True))
print("Perplexity no Interpolation: ", compute_perplexity(False))


Perplexity with Interpolation:  3490.97982265205
Perplexity no Interpolation:  1763.4299598123403


Here we can see the perplexity values are less for Listone smoothing without intepolation.
Limestone smoothing is not efficient two counts can be 0. so we need interpolation as it consider unigram. 
